In [1]:
import os
import json
import shutil
import lzma

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

import scipy.sparse as sps
from scipy.sparse import csr_matrix

import networkx as nx
import louvain
import igraph as ig

C:\Users\rdm42\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
<ipython-input-1-4bdac6f14819>:17: DeprecationWarning: This package has been superseded by the `leidenalg` package and will no longer be maintained. Please upgrade to the `leidenalg` package.
  import louvain


In [ ]:
def plot_degree_summary(Mwd=None, G=None):
    
    if G is None:
        dw = Mwd.sum(axis=1)
        print(f"Number of words: {Mwd.shape[0]}")
        print(f"Number of documents: {Mwd.shape[1]}")
    else:
        dw = pd.Series(dict(G.degree()))
        print(f"Number of words: {len(G)}")
        

    # Summary analysis of the network:
    gamma = 1 + dw.size/np.log(dw / dw.min()).sum()
    print(f"Network's gamma is {gamma:.2f}")


    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    plt.suptitle("Word degree summary", size=20)

    # Words' degrees

    n = 6
    s = dw.sort_values(ascending=False)
    custom_ticks = [min((s.size//n)*l,s.size-1) for l in range(n+1)]
    custom_labels = s.index[custom_ticks]

    axs[0].set_title("Degrees", size=16)
    axs[0].set_ylabel("Word count across documents", size=12)
    axs[0].semilogy(s)
    axs[0].set_xticks(custom_ticks)
    axs[0].set_xticklabels(custom_labels, rotation=30)


    # Words degree distribution

    k = np.unique(dw)
    pk = np.histogram(dw,np.append(k,k[-1]+1))[0]
    pk = pk/pk.sum()
    # Pk = 1-np.cumsum(pk)
    p_ideal = ((gamma-1)/k.min()) * ((k/k.min())**(-gamma))

    label = r'$\gamma$' + f" = {gamma:.2f}\n" + r'$k_{min}$' + f" = {k.min()}"
    axs[1].loglog(k, pk, 'o')
    axs[1].loglog(k, p_ideal,label=label)
    axs[1].set_title("Distribution", size=16)
    axs[1].set_xlabel("k", size=12)
    axs[1].set_ylabel("$p_k$", size=12)
    axs[1].legend(loc='upper right')

    plt.tight_layout()
    
    return gamma, fig, axs

In [ ]:
def clean_Mwd_matrix(Mwd,words,documents,thresh=2):

    # remove elements that are too central, e.g., #covid19
    not_wanted = Mwd.sum(axis=1) > Mwd.shape[1]/4
    text = "removing: " + ", ".join(words[not_wanted])
    words = words[~not_wanted]
    Mwd = Mwd[~not_wanted,:]

    # remove documents and words with fewer than 2 links
    while True:
        dim_old = Mwd.size
        
        # remove documents with less than 2 words
        wanted = Mwd.sum(axis=0) >= thresh
        Mwd = Mwd[:,wanted]
        documents = documents[wanted]

        # remove words in less than 2 documents
        wanted = Mwd.sum(axis=1) >= thresh
        text = text + ", " + ", ".join(words[~wanted])
        words = words[wanted]
        Mwd = Mwd[wanted,:]
        
        # exit criterion
        if (dim_old == Mwd.size): break

    print(text)
    return Mwd, words, documents

def logg(x):
    y = np.log(x)
    y[x==0] = 0
    return y



def probability_matrices(Mwd, equalik = True, tform = False):

    if equalik: # documents equally likely
        Pwd = Mwd/Mwd.sum(axis=0) #/ Mwd.shape[1]    ## Por qué aquí se divide entre el número de columnas???
        
    else: # documents proportional to their length
        Pwd = Mwd / Mwd.sum()                       ## Aquí la suma por columnas no da 1!!! entonces no es matriz estocástica
    # TF-IDF format
    
    if (tform):
        iw = -logg(np.sum(Mwd>0,axis=1).flatten()/Mwd.shape[1])
        Pwd = sps.diags(np.array(iw)[0])*Pwd # TF-IDF form
        Pwd = Pwd/Pwd.sum() # normalize, treat it as Pwd
        
    
    
    # Creo que aquí debe ser .dot ..............
    
    # words and document matrices
    pd = Pwd.sum(axis=0)
    Pww = (Pwd/pd).dot(Pwd.T)
    
    pw = Pwd.sum(axis=1)
    Pdd = (Pwd.T/pw).dot(Pwd)
    
    
    # joint words and document matrix - documents first
    Paa = sps.hstack((sps.csr_matrix((Pwd.shape[1],Pwd.shape[1])),Pwd.T))
    Paa = sps.vstack((Paa,sps.hstack((Pwd,sps.csr_matrix((Pwd.shape[0],Pwd.shape[0]))))))
    Paa = Paa/2.0
    
    return Pwd, Pww, Pdd, Paa

# Reading metadata

In [2]:
download_dir = os.path.join('..','0_data','downloaded_posts')
metadata_paths = [os.path.join(download_dir,f) for f in os.listdir(download_dir) if f.endswith('.json.xz')]

In [34]:
additional_data = []

for i,path in enumerate(metadata_paths):

    with lzma.open(path, 'rt') as file:
        data_dict = json.load(file)
    
    short = data_dict['node']['shortcode']
    user  = data_dict['node']['owner']['username']
    likes = data_dict['node']['edge_media_preview_like']['count']
    caption = data_dict['node']['edge_media_to_caption']['edges'][0]['node']['text']
    long = f"{os.path.split(path)[-1].split('.')[0]}.jpg"

    additional_data.append((short,user,likes,caption,long))

additional_data = pd.DataFrame(additional_data, columns=['Short','user','likes','caption','Long'])

additional_data.head(3)

,Short,user,likes,caption,Long
0,fNO52Nxluy,amenstyle,61,Valentina Vignali looks explosive in a preciou...,2013-10-08_12-40-36_UTC.jpg
1,lzQl_gRlnD,amenstyle,374,Regram @nimabenati #amenstyle #amen #fashion #...,2014-03-21_10-14-52_UTC.jpg
2,mrzAb-Rlkt,amenstyle,175,RG @nimabenati #amencouture amenstyle.com,2014-04-12_09-13-03_UTC.jpg


# Reading clean data

In [17]:
captions_path = os.path.join('..','0_data','text_clean.xlsx')
s = ' '

# text_col = 'clean_caption'
# text_col = 'clean_all'
text_col = 'clean_autocaption'

with pd.ExcelFile(captions_path) as xl:
    data = xl.parse(index_col=0) \
             .merge(additional_data, on='Short', how='left', validate='m:1') \
             .drop_duplicates() \
             .reset_index(drop=True)

# Inclusion of username in captions
for col in ['clean_caption','clean_autocaption','clean_all']:
    data[col] = data['user'] + ' ' + data[col]

# collection of words
words = np.unique([token
                   for tokens in data.dropna(subset=[text_col])[text_col].str.split()
                   for token in tokens])

documents = data[text_col].values

# words dictionary
words_dict = {wrd:i for i,wrd in enumerate(words)}

In [ ]:
# link
# post id
# description AI
# caption
# cleaned versions
# obj
# brand / user
# likes

# caterina.suitner@unipd.it

# Occurrence matrix $M_{wd}$

In [ ]:
# occurrence matrix for words
Mwd = csr_matrix((len(words_dict), data.shape[0]), dtype = np.int8).toarray()

for j,doc in data[text_col].dropna().items():
    for token in doc.split():
        Mwd[words_dict[token],j] += 1
        
        
# remove words that are used once or twice
select = Mwd.sum(axis=1) > 2
Mwd = Mwd[select,:]
words = words[select]

# remove documents that do not contain words
select = Mwd.sum(axis=0) > 0
Mwd = Mwd[:,select]
documents = documents[select]

# Further cleaning and probability matrices computation
Mwd, words, documents = clean_Mwd_matrix(Mwd,words,documents,thresh=2)
Pwd, Pww, Pdd, Paa = probability_matrices(Mwd, tform=False)

In [ ]:
Mwd_df = pd.DataFrame(Mwd,index=words,columns=documents)
Pww_df = pd.DataFrame(Pww,index=words,columns=words)

gamma, fig, axs = plot_degree_summary(Mwd_df)

# Objectification rate per word

In [ ]:
obj_docs = data.set_index(text_col) \
               .query("Objectify==1") \
               .index.drop_duplicates() \
               .intersection(Mwd_df.columns)


# Objectification rate for each word catalogue
# obj_rate = times the word is used in an objectifying post / times the word is used
obj_rate_cat = (Mwd_df[obj_docs].sum(axis=1) / Mwd_df.sum(axis=1)) \
               .to_frame().rename(columns={0:'obj_rate'}) \
               .reset_index(names='Id')

display(obj_rate_cat.head(3))
display(obj_rate_cat.describe().T)


sns.kdeplot(data=obj_rate_cat, x='obj_rate')
sns.rugplot(data=obj_rate_cat, x='obj_rate')

### La distribución luce multimodal

In [ ]:
links = Pww_df.melt(ignore_index=False, var_name='Target', value_name='Weight') \
              .reset_index() \
              .rename(columns={'index':'Source'})

links['temp_key'] = links.apply(lambda row: ''.join(sorted([row['Source'],row['Target']])),axis=1)

links.groupby('temp_key')['Weight'].std().unique()

In [ ]:
links = Pww_df.where((np.triu(np.ones(Pww_df.shape))).astype(np.bool_)) \
              .melt(ignore_index=False, var_name='Target', value_name='Weight') \
              .dropna() \
              .query("""Weight !=0""") \
              .reset_index() \
              .rename(columns={'index':'Source'})

sns.kdeplot(data=links, x='Weight')
sns.rugplot(data=links, x='Weight')
links.describe().T

In [ ]:
tol = links.Weight.quantile(0.05)
data_gephi = links.query("Weight >= @tol").sort_values('Weight')

In [ ]:
G = nx.from_pandas_edgelist(data_gephi, 'Source', 'Target', ['Weight'])
dw = pd.Series(dict(G.degree()))
_=plot_degree_summary(G=G)

In [ ]:
data_gephi.to_csv(f'data/processed/col_{text_col}__tol_{tol}__links.csv',
                  encoding='UTF-16',index=False)

pd.concat([data_gephi.Source,
           data_gephi.Target],
          ignore_index=True) \
  .drop_duplicates() \
  .sort_values() \
  .to_frame(name='Id') \
  .eval("Label=Id") \
  .merge(obj_rate_cat,how='left',on='Id',validate='m:1') \
  .to_csv(f'data/processed/col_{text_col}__tol_{tol}__nodes.csv',encoding='UTF-16',index=False)

In [ ]:
1024*5